# Linear Regression Consulting Project

Congratulations! You've been contracted by Hyundai Heavy Industries to help them build a predictive model for some ships. [Hyundai Heavy Industries](http://www.hyundai.eu/en) is one of the world's largest ship manufacturing companies and builds cruise liners.

You've been flown to their headquarters in Ulsan, South Korea to help them give accurate estimates of how many crew members a ship will require.

They are currently building new ships for some customers and want you to create a model and use it to predict how many crew members the ships will need.

Here is what the data looks like so far:

    Description: Measurements of ship size, capacity, crew, and age for 158 cruise
    ships.


    Variables/Columns
    Ship Name     1-20
    Cruise Line   21-40
    Age (as of 2013)   46-48
    Tonnage (1000s of tons)   50-56
    passengers (100s)   58-64
    Length (100s of feet)  66-72
    Cabins  (100s)   74-80
    Passenger Density   82-88
    Crew  (100s)   90-96
    
It is saved in a csv file for you called "cruise_ship_info.csv". Your job is to create a regression model that will help predict how many crew members will be needed for future ships. The client also mentioned that they have found that particular cruise lines will differ in acceptable crew counts, so it is most likely an important feature to include in your analysis! 

Once you've created the model and tested it for a quick check on how well you can expect it to perform, make sure you take a look at why it performs so well!

In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('lr_project').getOrCreate()

In [4]:
from pyspark.ml.regression import LinearRegression

In [5]:
df = spark.read.csv('cruise_ship_info.csv', inferSchema=True,header=True)

In [7]:
df.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)



In [11]:
for item in df.head(1)[0]:
    print(item)

Journey
Azamara
6
30.276999999999997
6.94
5.94
3.55
42.64
3.55


In [13]:
from pyspark.ml.linalg import Vectors

In [14]:
from pyspark.ml.feature import VectorAssembler

In [16]:
df.columns

['Ship_name',
 'Cruise_line',
 'Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'crew']

In [19]:
assembler = VectorAssembler(inputCols=['Tonnage','passengers','length','cabins','passenger_density'],outputCol='features')

In [20]:
output = assembler.transform(df)

In [21]:
output.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)
 |-- features: vector (nullable = true)



In [23]:
output.head(1)

[Row(Ship_name='Journey', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55, features=DenseVector([30.277, 6.94, 5.94, 3.55, 42.64]))]

In [24]:
final_data = output.select('features','crew')

In [25]:
final_data.show()

+--------------------+----+
|            features|crew|
+--------------------+----+
|[30.2769999999999...|3.55|
|[30.2769999999999...|3.55|
|[47.262,14.86,7.2...| 6.7|
|[110.0,29.74,9.53...|19.1|
|[101.353,26.42,8....|10.0|
|[70.367,20.52,8.5...| 9.2|
|[70.367,20.52,8.5...| 9.2|
|[70.367,20.56,8.5...| 9.2|
|[70.367,20.52,8.5...| 9.2|
|[110.238999999999...|11.5|
|[110.0,29.74,9.51...|11.6|
|[46.052,14.52,7.2...| 6.6|
|[70.367,20.52,8.5...| 9.2|
|[70.367,20.52,8.5...| 9.2|
|[86.0,21.24,9.63,...| 9.3|
|[110.0,29.74,9.51...|11.6|
|[88.5,21.24,9.63,...|10.3|
|[70.367,20.52,8.5...| 9.2|
|[88.5,21.24,9.63,...| 9.3|
|[70.367,20.52,8.5...| 9.2|
+--------------------+----+
only showing top 20 rows



In [26]:
train_data,test_data = final_data.randomSplit([0.7,0.3])

In [27]:
train_data.describe().show()

+-------+-----------------+
|summary|             crew|
+-------+-----------------+
|  count|              119|
|   mean|7.826890756302521|
| stddev|3.662515516921872|
|    min|             0.59|
|    max|             21.0|
+-------+-----------------+



In [28]:
test_data.describe().show()

+-------+------------------+
|summary|              crew|
+-------+------------------+
|  count|                39|
|   mean| 7.694358974358977|
| stddev|3.0075053439789716|
|    min|               0.6|
|    max|              13.6|
+-------+------------------+



In [29]:
lr = LinearRegression(labelCol='crew')

In [30]:
lr_model = lr.fit(train_data)

In [31]:
test_results = lr_model.evaluate(test_data)

In [32]:
test_results.residuals.show()

+--------------------+
|           residuals|
+--------------------+
|-0.01603227035337751|
|-0.09331262252389694|
|-0.38509021718686887|
|-0.00307174798577...|
|-0.08292978336163959|
|-3.14490036180004...|
|  0.7076357350262947|
|   1.123125465574141|
|  0.2662604899056831|
|  0.6606534970017846|
|-0.23033867546682796|
| -1.0881727230518603|
|-0.28303894901549675|
| -0.7440569217748525|
| -0.6673933833669494|
|-0.09955452847851554|
| 0.05481920890502945|
| -0.6924921954826653|
| -1.5750187316357636|
| 0.49783624752848965|
+--------------------+
only showing top 20 rows



In [33]:
test_results.rootMeanSquaredError

0.912484462597326

In [34]:
test_results.r2

0.9055245103603838

In [35]:
final_data.describe().show()

+-------+-----------------+
|summary|             crew|
+-------+-----------------+
|  count|              158|
|   mean|7.794177215189873|
| stddev|3.503486564627034|
|    min|             0.59|
|    max|             21.0|
+-------+-----------------+



In [36]:
from pyspark.sql.functions import corr

In [37]:
df.select(corr('crew','passengers')).show()

+----------------------+
|corr(crew, passengers)|
+----------------------+
|    0.9152341306065384|
+----------------------+



In [38]:
df.select(corr('crew','cabins')).show()

+------------------+
|corr(crew, cabins)|
+------------------+
|0.9508226063578497|
+------------------+

